# Homework set 4

Please **submit this Jupyter notebook through Canvas** no later than **Monday December 9, 9:00**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. On canvas there are hints about creating a nice pdf version.**

Before you hand in, please make sure the notebook runs, by running "Restart kernel and run all cells..." from the Kernel menu.

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

- Tycho Stam (13303147)
- Henry Zwart (15393879)



Run the following cell to import NumPy, Matplotlib. If anything else is needed you can import this yourself.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Exercise 1
N.B.1. you are to implement the methods yourself.

N.B.2. Tentative distribution of points is 2+1+2+2+2 points (plus 1 point makes 10).

Given a function $f$, let $T(f,a,b,m)$ denote the composite trapezoid rule with $m$ subintervals over the interval $[a,b]$. 

## (a)
Approximate the integral of $x^{-3}$ over $[a,b] = [ \frac{1}{10}, 100 ]$ by the composite trapezoid rule $T(f,a,b,m)$ for $m = 2^k$. Plot the absolute approximation error for different values of $k$. Find the smallest $k$ such that the exact error is less than $\epsilon = 10^{-3}$. Explain the slow convergence.

In [ ]:
FUNC_EVAL = 0


def f(x):
    """
    The function to be minimized.
    """
    global FUNC_EVAL

    try:
        FUNC_EVAL += len(x)
    except TypeError:
        FUNC_EVAL += 1

    return np.power(x, -3)

In [ ]:
def T(f, a, b, m, fa=None, fb=None, fmid=None):
    """
    Composite trapezoidal rule for a function f over the interval [a, b] with m subintervals.

    Takes optional keyword arguments fa, fb, fmid, the function evaluations at these
    points. If these are not supplied, they are calculated as necessary within the
    function. Otherwise the provided values are used. This allows elimination of
    unnecessary duplicate evaluations, for instance, in the composite midpoint rule.
    """
    h = (b - a) / m

    fa = fa or f(a)
    fb = fb or f(b)

    endpoint_value = (1 / 2) * (fa + fb)

    if m > 1:
        midpoint = (a + b) / 2
        fmid = fmid or f(midpoint)
        midpoint_value = fmid

        if m > 2:
            left_points = np.linspace(a + h, midpoint - h, (m // 2) - 1)
            right_points = np.linspace(midpoint + h, b - h, (m // 2) - 1)
            internal_value = f(left_points).sum() + f(right_points).sum()
        else:
            internal_value = 0
    else:
        midpoint_value = 0
        internal_value = 0

    return h * (endpoint_value + midpoint_value + internal_value)


def T_simple(f, a, b, m):
    """
    Simple trapezoidal rule for a function f over the interval [a, b] with m subintervals.
    """
    h = (b - a) / m
    x = np.linspace(a + h, b - h, m - 1)
    return h * ((f(a) / 2) + f(x).sum() + (f(b) / 2))

In [ ]:
a, b = 1 / 10, 100.0
true_value = 1 / 2 * (1 / 0.1**2 - 1 / 100**2)

k_list = np.arange(0, 20 + 1, dtype=int)
error_list = np.array([np.abs(T(f, a, b, 2**k) - true_value) for k in k_list])

epsilon = np.argmax(error_list < 10 ** (-3))
fig, axis = plt.subplots(1, 2, figsize=(10, 5))
axis[0].plot(k_list, error_list)
axis[0].set_xticks(k_list)
axis[0].set_xlabel(r"Number of sub-intervals ($\log_2$)")
axis[0].set_ylabel("Absolute approximation error")
axis[0].axvline(epsilon, color="red", label=r"$k_{\epsilon < 10^{-3}}$")
axis[0].legend()

x = np.linspace(a, 1, 1000)
y = f(x)
axis[1].plot(x, y)
axis[1].fill_between(x, y, alpha=0.5)
axis[1].set_xlabel("x")
axis[1].set_ylabel("f(x)")

plt.tight_layout()

The left graph shows the approximation error for different quanitites of subintervals,
with a red vertical line at the point where we achieve the desired error. The right 
graph shows the function evaluated on a small fraction of the integrated 
interval ($[0,1]$ versus $[0, 100]$). 

We see that for all but the smallest values of $x$, $f(x)$ is almost zero. As the graph 
is relatively flat in this region, the integrals evaluated on these subintervals is 
more accurate, and contributes less to the total integral, than subintervals closer to 
zero. 

The subintervals near zero -- in particular, the first subinterval -- are less 
straightforward. In order for the first subinterval to give an accurate estimate, the 
subinterval must be small enough that the rapid drop in the function appears 
approximately linear. It is evident from the graph that naive spacing for this first 
interval can easily result in a poor approximation.

To summarise, most of the integration interval is easy to evaluate accurately, except 
for near zero. Since subintervals are allocated uniformly, in order to evaluate this 
tricky region accurately, we must increase the subinterval density globally. This 
explains the large requirement on number of sub-intervals observed in the first plot.

## (b)

To improve the convergence rate of the above problem, we may use an adaptive strategy, as discussed in the book and the lecture. Consider the following formulas for approximate integration
$$I_1(f,a,b) = T(f,a,b,1)$$
$$I_2(f,a,b) = T(f,a,b,2).$$
Show, based on the precise error estimates for the trapezoid rule from the book/lecture that the error in $I_2$ can be estimated by a formula of the form 
$$E_2 = C (I_1 - I_2)$$
and determine the constant $C$ (if you can't find $C$, you may take $C = 0.5$).

### Solution

For an interval $[a,b]$, denote the midpoint by $m = \frac{a + b}{2}$, and let $h = 
(b - a)$. 

Evaluating a third-order Taylor expansion around $m$ at the endpoints, $a$ and $b$ 
gives: 

\begin{align*}
f(a) &= f(m) - f'(m)(h/2) + \frac{1}{2}f''(m)(h/2)^2 - \frac{1}{6}f^{(3)}(m)(h/2)^3 + O(h^4) \\
f(b) &= f(m) + f'(m)(h/2) + \frac{1}{2}f''(m)(h/2)^2 + \frac{1}{6}f^{(3)}(m)(h/2)^3 + O(h^4) \\
\end{align*}

Whose sum cancels odd-powered terms, producing:

$$
f(a) + f(b) = 2f(m) + \frac{1}{2}f''(m)(h/2)^2 + O(h^4)
$$

From the lectures, we obtain an expression for the integral of $f$ as a
fourth-order Taylor expansion around $m$:

$$
I(f) = f(m)h + \frac{1}{24}f''(m)h^3 + \frac{1}{1920}f^{(4)}(m)h^5 + O(h^7)
$$

$I_1$ and $I_2$ are expressed in terms of the composite trapezoid formula, as:

\begin{align*}
    I_1(f) &= \frac{h}{2}\left[f(a) + f(b)\right] \\
    I_2(f) &= \frac{h}{2}\left[\frac{f(a) + f(b)}{2} + f(m)\right]
\end{align*}

We first identify a simplifed form of the target difference, $I_1(f) - I_2(f)$, with 
sustitution of $f(a) + f(b)$ by the aforestated Taylor expansion:

\begin{align*}
I_1(f) - I_2(f) &= \frac{h}{2}\left[\frac{f(a) + f(b)}{2} - f(m)\right] \\
&= \frac{h}{2}\left[\frac{2f(m) + f''(m)(h/2)^2 + O(h^4)}{2} - f(m)\right] \\
&= \frac{1}{16}f''(m)h^3 + O(h^5)
\end{align*}

Finally, we show that the error in $I_2(f)$, denoted $E_2(f) = I_2(f) - I(f)$, is 
estimated by a scalar multiple of this difference:

\begin{align*}
I_2(f) - I(f) &= \frac{h}{2}\left[\frac{f(a) + f(b)}{2} + f(m)\right] - I(f) \\
&= \frac{h}{2}\left[\frac{2f(m) + \frac{1}{4}f''(m)h^2 + O(h^4)}{2} + f(m)\right] - I(f) \\
&= \frac{1}{24}f''(m)h^3 + O(h^5) \\
&= \frac{2}{3}\left[\frac{1}{16}f''(m)h^3\right] + O(h^5) \\
&= \frac{2}{3}\left[I_1(f) - I_2(f)\right] + O(h^5)
\end{align*}

So we see that $\frac{2}{3}(I_1(f) - I_2(f))$ is a reasonable approximation to $E_2(f)$.

Giving us $C = \frac{2}{3}$.

$\blacksquare$

## (c)
An adaptive strategy for computing the integral on an interval $[a,b]$ now is: Compute $I_2$ and $E_2$, and accept $I_2$ as an approximation when the estimated error $E_2$ is less or equal than a desired tolerance $\epsilon$.  Otherwise, apply the procedure to 
$\int_a^{\frac{b+a}{2}} f(x) \, dx$ and $\int_{\frac{b+a}{2}}^b f(x) \, dx$ with tolerances $\frac{\epsilon}{2}$.

Write a recursive python routine that implements the adaptive strategy.

Then apply this routine to the function $x^{-3}$ with $a, b, \epsilon$ as before. What is the exact error in the obtained approximation? 

In [ ]:
C = 2 / 3


def T_recursive(f, a, b, epsilon):
    """
    Recursive trapezoidal rule for a function f over the interval [a, b] with an error tolerance epsilon.
    """
    I_one = T(f, a, b, 1)
    I_two = T(f, a, b, 2)

    midpoint = (b + a) / 2
    if midpoint <= a or midpoint >= b:
        return I_two

    if abs(C * (I_one - I_two)) <= epsilon:
        return I_two

    right = T_recursive(f, midpoint, b, epsilon / 2)
    left = T_recursive(f, a, midpoint, epsilon / 2)

    return left + right

In [ ]:
def compute_time(func, args):
    """
    Compute the time taken to run a function with arguments.
    """
    start = time.time()
    func(*args)
    return time.time() - start

In [ ]:
a, b = 1 / 10, 100.0
true_value = 1 / 2 * (1 / 0.1**2 - 1 / 100**2)
EPSILON = 10**-3
k = 18
m = 2**k

approx_int_recur = T_recursive(f, a, b, EPSILON)
approx_int = T(f, a, b, m)

compute_time_recur = compute_time(T_recursive, (f, a, b, EPSILON))
compute_time_simple = compute_time(T, (f, a, b, m))

df = pd.DataFrame(
    {
        "Method": ["Recursive", "Simple", "True value"],
        "Approximation": [approx_int_recur, approx_int, true_value],
        "Error": [
            np.abs(approx_int_recur - true_value),
            np.abs(approx_int - true_value),
            "",
        ],
        "Time": [compute_time_recur, compute_time_simple, ""],
    }
)
df

The error with the recursive function is smaller than the non-recursive one, at the 
cost of a longer compute time. We note that the error resulting from the recursive 
function is significantly lower than the required threshold 
($1.7\cdot 10^{-4} < 10^{-3}$).

## (d)

Modify the code of (c) so that the number of function evaluations is counted. Optimize your implementation such that no unnecessary function evaluations are performed.

Compare the number of function evaluations used in the adaptive strategy of (c) with the result of (a). 
(*Hint*: To count the number of function evaluations, you may use a global variable.)


### Solution

Unnecessary function evaluations exist in the following places:
- $I_1$ and $I_2$ both evaluate $f(a)$ and $f(b)$
- $f(a)$, $f(mid)$ and $f(b)$ are evaluated by the left and right recursive calls, as the new endpoints.

We eliminate these calls by modifying:
- The recursive function to take optional $f(a)$ and $f(b)$ as parameters
- The trapezoid rule function to take $f(a)$, $f(b)$, and $f(mid)$ (when used with two subintervals).

In [ ]:
fa, fb = f(a), f(b)


def T_recursive_optimised(f, a, b, epsilon, fa=None, fb=None):
    """
    Optimised recursive trapezoidal rule for a function f over the interval [a, b] with an error tolerance epsilon.
    """
    fmid = f((a + b) / 2)
    I_one = T(f, a, b, 1, fa=fa, fb=fb)
    I_two = T(f, a, b, 2, fa=fa, fmid=fmid, fb=fb)
    C = 6 / 8

    midpoint = (b + a) / 2
    if midpoint <= a or midpoint >= b:
        return I_two

    if abs(C * (I_one - I_two)) <= epsilon:
        return I_two

    return T_recursive_optimised(
        f, a, midpoint, epsilon / 2, fa, fmid
    ) + T_recursive_optimised(f, midpoint, b, epsilon / 2, fmid, fb)

In [ ]:
def func_eval(func, args):
    """
    Count the number of function evaluations.
    """
    global FUNC_EVAL
    FUNC_EVAL = 0
    func(*args)
    return FUNC_EVAL

In [ ]:
eval_val = np.zeros(4)
eval_val[0] = func_eval(T_recursive, (f, a, b, EPSILON))
eval_val[1] = func_eval(T, (f, a, b, m))
eval_val[2] = func_eval(T_recursive_optimised, (f, a, b, EPSILON, fa, fb))

compute_time_recur_opt = compute_time(T_recursive_optimised, (f, a, b, EPSILON, fa, fb))

df = pd.DataFrame(
    {
        "Method": ["Recursive", "Simple", "Recursive (opt)", "True value"],
        "Approximation": [approx_int_recur, approx_int, approx_int_recur, true_value],
        "Error": [
            np.abs(approx_int_recur - true_value),
            np.abs(approx_int - true_value),
            np.abs(approx_int_recur - true_value),
            "",
        ],
        "Time": [compute_time_recur, compute_time_simple, compute_time_recur_opt, ""],
        "Function evaluations": eval_val,
    }
)
df

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax.bar(
    [
        "Recursive",
        "Simple",
        "Recursive (opt)",
    ],
    eval_val[:-1],
    color=["tab:blue", "tab:orange", "tab:red"],
)

ax.set_xlabel("Algorithm")
ax.set_ylabel(r"Number of $f(x)$ evaluations")

fig.tight_layout()
plt.show()

As seen in the image above the _Simple_ method is the least efficient, requiring significantly more evaluations of $f(x)$ than the others. The _Recursive_ algorithm improves on this, but the _Recursive (opt)_ method is by far the most efficient, needing the fewest evaluations of $f(x)$.
In compute time the _Simple_ algorithm is still the fastest, even over the _Recursive (opt)_ method, but results in greater error. Whereby the _Recursive (opt)_ is faster than the _Recursive_ method and gives equivalent error.

## (e)
In the course of executing the recursive procedure, some subintervals are refined (split into two subintervals) while others aren't as a result of the choices made by the algorithm. It turns out that the choices made by this algorithm are not always optimal. Other algorithms, that decide in a different way which subinterval needs to be refined, may be more efficient in the sense that they require less function evaluations (while using the same formulas for the approximate integral and the approximate error associated with a subinterval).

Can you explain why this is the case? Devise an alternative, non-recursive algorithm that addresses this issue and should to lead a more efficient integral computation. Describe your approach and algorithm in about 5 to 10 sentences (bullet points).


### Solution

The recursive approach is short-sighted, and requires uniform error across subintervals.
This is evident in the fact that each recursive call provides an updated error 
requirement of $\epsilon/2$. In reality, small total error across the integral $[a,b]$ 
does not require uniformly low error in subintervals. We may choose to allow larger 
error in some areas, in which the integral is difficult to calculate, and offset this 
with lower error in others, where the integral is relatively easier. 

For instance, in the function studied here, we observed earlier in the notebook that 
most subintervals on the interval $[0.1, 100]$ contribute very little to the total 
integral. Consequentially, their contribution to the total error is also small. Despite 
this, the recursive algorithm assumes that the intervals $[0.1,50.05]$ and $[50.05,100]$ 
contribute equally to the error, and places identically strict error requirements 
on their recursive calls.

We describe the following algorithm which attempts to address this issue:
1. Divide the interval $[a,b]$ into two subintervals.
2. Estimate the integral on each subinterval.
3. Estimate the error on each subinterval, and the resulting total error.
4. While the total error exceeds a pre-defined threshold:
    1. Subdivide the subinterval with the largest error.
    2. Estimate the integrals on each of the two new subintervals.
    3. In the total (summed) integral estimate, replace the contribution of the original subinterval with those of the new ones.
    4. Estimate the error in each of the two new subintervals.

This algorithm retains a global view of the estimation error on the whole interval 
$[a,b]$. In the context of the function studied in this notebook, we would expect to 
see less work spent improving estimates in the interval $[1,100]$, and more in the 
subintervals close to zero. Unlike the recursive algorithm; however, this algorithm 
does not require that each individual subinterval has low-error -- only that the total 
sum does. This results in fewer function evaluations to achieve the same error bound.